In [1]:
import random
import time

In [2]:
class Node:
    def __init__(self, key, value):
        self.data = (key, value)
        self.next = None
        self.prev = None


class DoublyLinkedList:
    def __init__(self):
        self.head = None
        self.tail = None

    def push(self, key, value):
        new_node = Node(key, value)
        new_node.next = self.head
        if self.head:
            self.head.prev = new_node
        else:
            self.tail = new_node
        self.head = new_node
        return new_node

    def remove(self, node):
        if node.prev:
            node.prev.next = node.next
        else:
            self.head = node.next
        if node.next:
            node.next.prev = node.prev
        else:
            self.tail = node.prev
        node.prev = None
        node.next = None

    def move_to_front(self, node):
        if node != self.head:
            self.remove(node)
            node.next = self.head
            self.head.prev = node
            self.head = node

    def remove_last(self):
        if self.tail:
            last = self.tail
            self.remove(last)
            return last
        return None


class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache = {}
        self.list = DoublyLinkedList()

    def get(self, key):
        if key in self.cache:
            node = self.cache[key]
            self.list.move_to_front(node)
            return node.data[1]
        return -1

    def put(self, key, value):
        if key in self.cache:
            node = self.cache[key]
            node.data = (key, value)
            self.list.move_to_front(node)
        else:
            if len(self.cache) >= self.capacity:
                last = self.list.remove_last()
                if last:
                    del self.cache[last.data[0]]
            new_node = self.list.push(key, value)
            self.cache[key] = new_node

    def invalidate_range(self, index):
        keys_to_remove = []

        for key in self.cache.keys():
            left, right = key
            if left <= index <= right:
                keys_to_remove.append(key)

        for key in keys_to_remove:
            del self.cache[key]

    def clear(self):
        self.cache.clear()

In [3]:
def make_queries(n, q, hot_pool=30, p_hot=0.95, p_update=0.03):
    hot = [(random.randint(0, n//2), random.randint(n//2, n-1))
           for _ in range(hot_pool)]
    queries = []
    for _ in range(q):
        if random.random() < p_update:        # ~3% запитів — Update
            idx = random.randint(0, n-1)
            val = random.randint(1, 100)
            queries.append(("Update", idx, val))
        else:                                 # ~97% — Range
            if random.random() < p_hot:       # 95% — «гарячі» діапазони
                left, right = random.choice(hot)
            else:                             # 5% — випадкові діапазони
                left = random.randint(0, n-1)
                right = random.randint(left, n-1)
            queries.append(("Range", left, right))
    return queries

In [4]:
def range_sum_no_cache(array, left, right):
    return sum(array[left:right+1])


def update_no_cache(array, index, value):
    array[index] = value


def range_sum_with_cache(array, left, right):
    key = (left, right)

    cached_value = cache.get(key)

    if cached_value != -1:
        return cached_value

    result = sum(array[left : right + 1])
    cache.put(key, result)
    return result


def update_with_cache(array, index, value):
    array[index] = value
    cache.invalidate_range(index)

In [5]:
def run_without_cache(array, queries):
    arr = array.copy()

    start_time = time.time()

    for query in queries:
        if query[0] == "Range":
            _, left, right = query
            range_sum_no_cache(arr, left, right)
        elif query[0] == "Update":
            _, index, value = query
            update_no_cache(arr, index, value)

    end_time = time.time()
    
    return end_time - start_time

In [6]:
def run_with_cache(array, queries):
    arr = array.copy()
    cache.clear()

    start_time = time.time()

    for query in queries:
        if query[0] == "Range":
            _, left, right = query
            range_sum_with_cache(arr, left, right)
        elif query[0] == "Update":
            _, index, value = query
            update_with_cache(arr, index, value)

    end_time = time.time()
    
    return end_time - start_time

In [7]:
n = 100_000
q = 50_000

cache = LRUCache(capacity=1000)
array = [random.randint(1, 100) for _ in range(n)]
queries = make_queries(n, q)

In [8]:
time_no_cache = run_without_cache(array, queries)
time_with_cache = run_with_cache(array, queries)

speedup = time_no_cache / time_with_cache
print(f"No cache: {time_no_cache:.2f}")
print(f"LRU-cache: {time_with_cache:.2f} (speed up ×{speedup:.1f})")

No cache: 13.82
LRU-cache: 5.39 (speed up ×2.6)
